<a href="https://colab.research.google.com/github/Bentley97/NLU_First_assignment/blob/main/FirstAssignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**FIRST ASSIGNMENT**

Student:
*   Name: Luca
*   Surname: Bentivoglio
*   Student number: 221246




**REQUIREMENTS**

If you open the notebook, is sufficient to run the first block of code to import spaCy, define a sentence for tests and load the dependency parsing model.
Then you can run each function with its test calls.

In [4]:
import spacy

test_sentence = "I saw a man with a telescope."

nlp = spacy.load('en_core_web_sm')

**FUNCTION 1**

In [ ]:
def extract_dependency_path(sentence):
  doc = nlp(sentence)

  paths = []

  for token in doc:
    token_path = []

    while token != token.head:  #we can use also (token != token.sent.root)
      token_path.insert(0, token.text)
      #token_path.insert(0, "goes with dep '"+token.dep_+"' in")
      token_path.insert(0, "--"+token.dep_+"->")
      token = token.head

    token_path.insert(0, token.text)
    #token_path.insert(0, "goes with dep '"+token.dep_+"' in")
    token_path.insert(0, "--"+token.dep_+"->")
    token_path.insert(0, "ROOT")
    paths.append(token_path)
  
  return paths


In [ ]:
### TEST ###
dep_paths = extract_dependency_path(test_sentece)

for dep_path in dep_paths:
  print("The dependency path for token '{}'\tis:\t{}".format(dep_path[-1],dep_path).expandtabs(15))


The dependency path for token 'I'            is:            ['ROOT', '--ROOT->', 'saw', '--nsubj->', 'I']
The dependency path for token 'saw'          is:            ['ROOT', '--ROOT->', 'saw']
The dependency path for token 'a'            is:            ['ROOT', '--ROOT->', 'saw', '--dobj->', 'man', '--det->', 'a']
The dependency path for token 'man'          is:            ['ROOT', '--ROOT->', 'saw', '--dobj->', 'man']
The dependency path for token 'with'         is:            ['ROOT', '--ROOT->', 'saw', '--dobj->', 'man', '--prep->', 'with']
The dependency path for token 'a'            is:            ['ROOT', '--ROOT->', 'saw', '--dobj->', 'man', '--prep->', 'with', '--pobj->', 'telescope', '--det->', 'a']
The dependency path for token 'telescope'    is:            ['ROOT', '--ROOT->', 'saw', '--dobj->', 'man', '--prep->', 'with', '--pobj->', 'telescope']


**FUNCTION 2**

In [ ]:
def extract_subtrees(sentence):
  doc = nlp(sentence)

  result = []
  
  for token in doc:
    subtree_list = []
    for element in token.subtree:
      if element != token:
        subtree_list.append(element)
    result.append([token, subtree_list])

  return result  #returns a list of elements, each composed by the token and the list of its dependents in its subtree(without himself) ordered w.r.t. sentence order  (usare elenco puntato per spiegare meglio)

In [ ]:
subtrees = extract_subtrees(test_sentence)

for t,s in subtrees:
  #print(s)
  print("Token: {}\t{}".format(t,s).expandtabs(20))

Token: I            []
Token: saw          [I, a, man, with, a, telescope]
Token: a            []
Token: man          [a, with, a, telescope]
Token: with         [a, telescope]
Token: a            []
Token: telescope    [a]


**FUNCTION 3**

In [ ]:
def is_equal(tokenList, stringList):  #check equality between elements of a list of tokens and a list of strings
  if len(tokenList) != len(stringList):
    return False

  for i in range(len(tokenList)):
    if tokenList[i].text != stringList[i]:
      return False
  
  return True

def is_subtree(sentence, segment): #segment => ordered list of words from a sentence
  doc = nlp(sentence)

  for token in doc:
    subtree = token.subtree
    if(is_equal(list(subtree), segment)):
        return True

  return False

In [ ]:
input_segment = ["a","man","with","a","telescope"]
input_segment2 = ["a","man"]
input_segment3 = ["with","a","telescope"]


print("Sentence:\t{}".format(test_sentence))
print("Segment:\t{}".format(input_segment3))
print(is_subtree(test_sentence, input_segment3))


#print(list(input_segment)==list(input_segment3))
#print(type(input_segment))
#print(is_subtree(test_sentence, input_segment))
#print(is_subtree(test_sentence, input_segment2))

Sentence:	I saw a man with a telescope
Segment:	['with', 'a', 'telescope']
True


**FUNCTION 4**

In [8]:
def get_head(span):     #identify head of a span
  doc = nlp(span)

  return [token for token in doc if token.head == token][0]   # the condition can be changed with (token.dep_ == 'ROOT')      we can use also Span object of spacy


In [9]:
span1 = "Give it to me! Right now."   #to test return single word 
print(get_head(span1))

span2 = "boowling water bad under down mountain"
print(get_head(span2))

Give
water


**FUNCTION 5**

In [24]:
def extract_spans(sentence):
  doc = nlp(sentence)

  span_dict = {
      'nsubj':[],
      'dobj':[],
      'iobj':[]
  }

  for token in doc:
    if token.dep_ == 'nsubj' or token.dep_ == 'dobj' or token.dep_ == 'iobj':
      for descendant in token.subtree:                    # instead of cycle on elements of the subtree we can use left_edge and right_edge attributes that gives directly first and last element of the token's subtree
        span_dict[token.dep_].append(descendant.text)     #span_dict[token.dep_] = doc[token.left_edge.i : token.right_edge.i+1]
  
  return span_dict


In [25]:
sentence = "I saw a man with a telescope."

spans = extract_spans(sentence)

for item in spans.items():
  print(item)

('nsubj', ['I'])
('dobj', ['a', 'man', 'with', 'a', 'telescope'])
('iobj', [])
